In [2]:
from src.data.data_wrangling import *
from src.data.load_data_reviews_kaggle import *
from src.data.load_data import *
from src.data.load_data_non_commercial_imdb import *
from src.data.predict_emotions import *
from src.scripts.scrap_date import *

# Make sure that if any dependencies changes it will be reflected in the notebook (From the ML course)
%load_ext autoreload
%autoreload 2

In [3]:
# Load movie metadata
df_movies_metadata, df_movies_language, df_movies_countries, df_movies_genres = load_and_clean_movies_df()

In [3]:
# Load movie metadata
# df_movies_metadata, df_movies_language, df_movies_countries, df_movies_genres = load_and_clean_movies_df()

# Load movie reviews from kaggle
#df_reviews_kaggle = load_movie_reviews_kaggle(df_movies_metadata)

# Load movie average reviews from the non-commercial IMDB
#imdb_average_reviews = load_imdb_average_reviews(df_movies_metadata)

# Compute plot emotions
#predict_emotions_to_csv(df_movies_metadata)

In [4]:
# # Predict emotions

# predict_emotions_to_csv(df_movies_metadata,
#                         column_to_predict='plot', 
#                         csv_path='plot_emotions.csv')

# # Merging df_movies with plot_emotions.csv
# df_movies_with_emotions = merge_df_movies_with_emotions_csv(df_movies_metadata, csv_path='plot_emotions.csv')

In [ ]:
# We scrap our years and months data using:
# scrap_years_months_movies(df_movies_metadata)

# Then we load the resulting csv and combine both data (from both the original dataset and scrapped data)
# get_final_dates(pd.read_csv("dates_scraped.csv"))